In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
from tqdm import tqdm
HOME = expanduser("~")
import os, sys
import cv2
import skimage.exposure
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52'
INPUT = os.path.join(DIR, 'preps', 'CH1', 'thumbnail')
CLEANED = os.path.join(DIR, 'preps', 'CH1', 'thumbnail_cleaned')
MASKED = os.path.join(DIR, 'preps', 'thumbnail_masked')
OUTPUT = os.path.join(HOME, 'fixed')
files = sorted(os.listdir(INPUT))

In [3]:
PATH = '/home/eddyod/programming/pipeline_utility'
sys.path.append(PATH)
from utilities.utilities_mask import linnorm, fix_with_fill, find_contour_count, pad_with_black, fill_spots

Connecting dklab@192.168.1.12:3306


In [4]:
    badi = [35,
            36,
            37,
            39,
            40,
            41,
            42,
            60,
            61,
            62,
            63,
            64,
            65,
            68,
            69,
            70,
            71,
            72,
            73,
            74,
            76,
            112,
            113,
            114,
            115,
            131,
            158,
            159,
            160,
            161,
            162,
            163,
            164,
            165,
            166,
            167,
            172,
            176,
            177,
            178,
            179,
            180,
            181,
            182,
            183,
            184,
            185,
            186,
            187,
            188,
            190,
            232,
            233,
            234,
            235,
            255,
            294,
            337,
            338,
            339,
            340,
            341,
            342,
            343,
            344,
            345,
            346,
            347,
            348,
            349,
            350,
            391,
            400,
            401,
            405,
            406,
            407,
            408,
            409,
            410,
            411,
            412,
            413,
            414,
            415,
            416,
            417,
            418,
            419,
            420,
            421,
            422,
            423,
            424,
            425,
            426,
            427,
            428,
            429,
            430,
            431,
            432,
            433,
            434,
            435]


In [5]:
bads = ['{}.tif'.format(str(i).zfill(3)) for i in badi]

In [24]:
# get oriented for comparison
img_inputs = []
img_outputs = []
file_inputs = []
masks = []
fixeds = []
limit = 250
dt = np.uint8
clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(8, 8))

for i, file in enumerate(tqdm(bads)):
    infile = os.path.join(INPUT, file)
    img = io.imread(infile)
    img_inputs.append(img)
    normed = linnorm(img, 25000, np.uint16)
    img_outputs.append(normed)
    mask = fix_with_fill(img, limit, dt)
    contours = find_contour_count(mask)
    
    file_inputs.append((file,contours))
    masks.append(mask)
    
    fixed = cv2.bitwise_and(img, img, mask=mask)
    fixed = clahe.apply(fixed.astype(np.uint16))
    #fixed = linnorm(fixed, 25000, np.uint16)

    fixeds.append(fixed)
    
    normed_outpath = os.path.join(OUTPUT, 'normed_{}'.format(file))
    cv2.imwrite(normed_outpath, normed.astype('uint16'))
    
    mask_outpath = os.path.join(OUTPUT, 'mask_{}'.format(file))
    cv2.imwrite(mask_outpath, mask.astype('uint8'))
    
    fixed_outpath = os.path.join(OUTPUT, 'fixed_{}'.format(file))
    cv2.imwrite(fixed_outpath, fixed.astype('uint16'))



100%|██████████| 105/105 [00:19<00:00,  5.48it/s]


In [ ]:

ncols = 3
fig, ax = plt.subplots(nrows=len(masks), ncols=ncols, sharex=False, sharey=False)
plt.style.use('classic')
i = 0
for fileout,name, mask, fixed in zip(img_outputs, file_inputs, masks, fixeds):
    ax[i,0].set_title('normed {}'.format(name))
    ax[i,0].imshow(fileout, cmap="gray")
    ax[i,1].set_title('mask ')
    ax[i,1].imshow(mask, cmap="gray")
    ax[i,2].set_title('mask ')
    ax[i,2].imshow(fixed, cmap="gray")
    i += 1
# swap the 10 and the six below if the images are wider than height
fig.set_size_inches(np.array([6*ncols, 4 * len(masks)]), forward=True)
plt.show()    
